In [1]:
%pip install speechbrain

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
import os
import numpy as np
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error

c:\Users\Abhinav\Anaconda\envs\speech_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
torchvision is not available - cannot save figures
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [3]:
classifier = EncoderClassifier.from_hparams(source="C://Users/Abhinav/Fatigue/spkrec-ecapa-voxceleb/")

In [4]:
dir_path = "C:\\Users\\Abhinav\\vf_data"
embeddings_list = []
labels_list = []
for filename in os.listdir(dir_path):
    if filename.endswith(".wav"):
        file_path = os.path.join(dir_path, filename)
        signal, fs = torchaudio.load(file_path)
        embeddings = classifier.encode_batch(signal)
        embeddings_list.append(embeddings)
        if "fatigue" in filename:
            labels_list.append(1)  # 1 represents a fatigued recording
        else:
            labels_list.append(0)  # 0 represents a non-fatigued recording




c:\Users\Abhinav\Anaconda\envs\speech_env\lib\site-packages\torch\functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:867.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [5]:
embeddings_matrix = np.concatenate(embeddings_list, axis=0)
print(embeddings_matrix.shape)
reshaped_embeddings = embeddings_matrix.reshape(embeddings_matrix.shape[0], -1)

(8, 1, 192)


In [6]:
perplexity_value = 5

tsne = TSNE(n_components=2, perplexity=perplexity_value)
tsne_embeddings = tsne.fit_transform(reshaped_embeddings)
print(len(reshaped_embeddings))

8


In [7]:
weights = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
print(len(weights))
padding = len(weights) - 1
padded_embeddings = np.pad(tsne_embeddings, ((padding, 0), (0, 0)), mode='constant')
smoothed_embeddings = np.apply_along_axis(lambda m: np.convolve(m, weights, mode='valid'), axis=0, arr=padded_embeddings)
print(len(smoothed_embeddings))

5
8


In [8]:
target_rms = 0.85  # Adjust this value as per your requirement

# Calculate the RMS of the smoothed_embeddings
rms = np.sqrt(np.mean(np.square(smoothed_embeddings), axis=0))

# Calculate the scale factor to normalize the embeddings
scale_factor = target_rms / rms

# Apply the scale factor to normalize the embeddings
normalized_embeddings = smoothed_embeddings * scale_factor

print("Hello can i branch")

Hello can i branch


In [9]:
random_state = 42 

X_train, X_test, y_train, y_test = train_test_split(normalized_embeddings, labels_list, test_size=0.20, random_state=random_state)



In [10]:
C = 1.0  # Regularization parameter
kernel = 'rbf'  # Kernel function (e.g., 'linear', 'rbf', 'poly', etc.)
gamma = 'scale'  # Kernel coefficient (e.g., 'scale', 'auto', float)

#Fit SVM on data


svm = SVC(C=C, kernel=kernel, gamma=gamma, random_state=random_state)
svm.fit(X_train, y_train)

SVC(random_state=42)

In [11]:
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 0.00%


In [12]:
linear_reg = LinearRegression()
ridge_reg = Ridge(alpha=1.0, random_state=random_state)
lasso_reg = Lasso(alpha=0.1, random_state=random_state)
models = [linear_reg, ridge_reg, lasso_reg]
model_names = ["Linear Regression", "Ridge Regression", "Lasso Regression"]
for model, model_name in zip(models, model_names):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    print("{} - Mean squared error: {:.2f}".format(model_name, mse))

Linear Regression - Mean squared error: 0.40
Ridge Regression - Mean squared error: 0.42
Lasso Regression - Mean squared error: 0.44
